In [5]:
import pandas as pd
import numpy as np
from eval_utils import compute_accuracy, get_summary

In [59]:
!ls "./results/zero-shot/pangea/"

full_accuracy.json    pangea-formatted.json pangea-raw.json


In [65]:
result_path = "./results/closed/aya-vision/aya-formatted.json"
full_acc = pd.read_json(compute_accuracy(result_path))

Accuracy saved in results/closed/aya-vision/full_accuracy.json


In [66]:
summary = get_summary(full_acc, "language")
summary

,language,accuracy,valid_acc,valid_count,total_questions,none_count,correct,num_lang,languages
0,Arabic,0.337696,0.341270,378,382,4,129.0,1,{Arabic}
1,Bengali,0.203750,0.301294,541,800,259,163.0,1,{Bengali}
2,Croatian,0.216049,0.219436,319,324,5,70.0,1,{Croatian}
3,Dutch; Flemish,0.400786,0.401180,1017,1018,1,408.0,1,{Dutch; Flemish}
4,English,0.421990,0.423290,1623,1628,5,687.0,1,{English}
5,French,0.366142,0.366142,762,762,0,279.0,1,{French}
6,German,0.433518,0.433518,722,722,0,313.0,1,{German}
7,Hindi,0.336161,0.353795,1792,1886,94,634.0,1,{Hindi}
8,Hungarian,0.277679,0.279676,1112,1120,8,311.0,1,{Hungarian}
9,Lithuanian,0.388235,0.391691,674,680,6,264.0,1,{Lithuanian}


In [67]:
accuracy = np.mean(summary['accuracy'])
valid_acc = np.mean(summary['valid_acc'])
rate = 1 - sum(summary['valid_count'])/sum(summary['total_questions'])
print(f"Accuracy: {accuracy*100}")
print(f"Valid Accuracy: {valid_acc*100}")
print(f"Failure Rate: {rate*100}")
print(f"Total questions:{np.sum(summary['total_questions'])}")

Accuracy: 33.084666500829044
Valid Accuracy: 35.15402125380515
Failure Rate: 6.216823681316052
Total questions:20911


In [28]:
def print_column(summary, category):
    for value in summary[category]:
        print(np.round(value, decimals=1))
        
print_column(summary, "none_count")

3
46
25
7
0
18
0
100
41
6
10
66
6
56
92
2
2
12


In [ ]:
results = None
for model_name, p in result_paths.items():
    full_acc = pd.read_json(p)
    summary = get_summary(full_acc, "language")
    if results is None:
        results = summary[["language", "none_count"]]
        results = results.rename(columns={"none_count": model_name})
    else:
        results[model_name] = summary['none_count']

In [68]:
#results = None
if results is None:
    results = summary[["language", "none_count"]]
    results = results.rename(columns={"none_count": "qwen2.5-vl-7b"})
else:
    results["aya"] = summary['none_count']

In [71]:
results

,language,qwen2.5-vl-7b,molmo,gpt,claude,gemini,pangea,aya
0,Arabic,0,0,32,0,2,27,4
1,Bengali,0,0,20,1,7,32,259
2,Croatian,5,0,18,2,6,12,5
3,Dutch; Flemish,0,0,57,3,35,2,1
4,English,0,0,279,340,9,14,5
5,French,0,0,33,0,14,3,0
6,German,0,0,10,0,0,0,0
7,Hindi,6,12,115,1,18,176,94
8,Hungarian,24,0,75,2,30,23,8
9,Lithuanian,103,1,15,1,0,2,6


In [6]:
import json
with open("./results/closed/aya-vision/aya-formatted.json", "r") as f:
    data = json.load(f)

In [7]:
missing = []
for sample in data:
    if sample['prediction'] not in [0,1,2,3]:
        missing.append(sample)

In [ ]:
for k in data[0].keys():
    if "prediction" in k:
        print(k)

prediction_by_gemini-1.5-pro
prediction_by_claude-3-5-sonnet-latest
prediction_by_gpt-4o
prediction_by_molmo
prediction_by_pangea
prediction_by_qwen2.5-7b


In [42]:
missing = []
for sample in data:
    if sample['prediction_by_pangea'] not in [0,1,2,3]:
        missing.append(sample)